In [54]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import zipfile
import io
import pandas as pd
import math
from IPython.display import display
from collections import defaultdict

zipped_data_path = "../data/clean_data/class-competition-cleaned.zip"

dataframes = defaultdict(pd.DataFrame)
with zipfile.ZipFile(zipped_data_path, "r") as zip:
    for filename in zip.namelist():
        if filename.endswith(".csv"):
            with zip.open(filename) as f:
                dataframes.update({ filename : pd.read_csv(io.TextIOWrapper(f)) })

                # Lets take a look at the files
                print(f"FILE: {filename}")
                # If you want to see file info uncomment this:
                # display(dataframes[filename].info())
                # display(dataframes[filename].head())

FILE: train_call_type_A.csv
FILE: train_call_type_B.csv
FILE: train_call_type_C.csv
FILE: test_public.csv


In [55]:
train_data = dataframes["train_call_type_B.csv"]
test_data  = dataframes["test_public.csv"]
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [56]:
# Split the START_LOCATION into two new columns for latitude and longitude for train set
for i, j in train_data['START_LOCATION'].items():
    split = j.split(',')
    train_data.at[i, 'START_LAT'] = split[0]
    train_data.at[i, 'START_LONG'] = split[1]
train_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,POLYLINE,TRAVEL_TIME,START_LOCATION,MON_sin,MON_cos,DAY_sin,DAY_cos,HR_sin,HR_cos,WK_sin,WK_cos,YR_2013,START_LAT,START_LONG
0,1372637303620000596,B,NaN,7.0,20000596,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,"41.1599801853,-8.64198392478",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1599801853,-8.64198392478
1,1372638481620000403,B,NaN,28.0,20000403,"[[-8.584263,41.163156],[-8.584695,41.163003],[...",960,"41.1632224305,-8.58404677278",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1632224305,-8.58404677278
2,1372639960620000309,B,NaN,38.0,20000309,"[[-8.60418,41.160969],[-8.603874,41.1615],[-8....",285,"41.1607148883,-8.60424608207",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1607148883,-8.60424608207
3,1372639635620000178,B,NaN,52.0,20000178,"[[-8.613243,41.154444],[-8.612811,41.153733],[...",255,"41.1549650972,-8.61321698848",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1549650972,-8.61321698848
4,1372639973620000686,B,NaN,23.0,20000686,"[[-8.61255,41.146128],[-8.612199,41.146074],[-...",285,"41.1460158298,-8.61257471887",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1460158298,-8.61257471887


In [57]:
# Use only data points with CALL_TYPE B
test_data = test_data[test_data['CALL_TYPE'] == 'B']
# Split the START_LOCATION into two new columns for latitude and longitude for test set
for i, j in test_data['START_LOCATION'].items():
    split = j.split(',')
    test_data.at[i, 'START_LAT'] = split[0]
    test_data.at[i, 'START_LONG'] = split[1]
test_data.head()

/var/folders/j4/rs0wkwns5_34kxc5nfm8vbh40000gn/T/ipykernel_6085/2665617917.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.at[i, 'START_LAT'] = split[0]
/var/folders/j4/rs0wkwns5_34kxc5nfm8vbh40000gn/T/ipykernel_6085/2665617917.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.at[i, 'START_LONG'] = split[1]


,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,START_LOCATION,MON_sin,MON_cos,DAY_sin,DAY_cos,HR_sin,HR_cos,WK_sin,WK_cos,YR_2013,START_LAT,START_LONG
0,T1,B,NaN,15.0,20000542,"41.1486275073,-8.58587660305",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1486275073,-8.58587660305
1,T2,B,NaN,57.0,20000108,"41.1457188058,-8.61070701502",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1457188058,-8.61070701502
2,T3,B,NaN,15.0,20000370,"41.1486275073,-8.58587660305",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1486275073,-8.58587660305
3,T4,B,NaN,53.0,20000492,"41.1412081961,-8.61401226127",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1412081961,-8.61401226127
4,T5,B,NaN,18.0,20000621,"41.1483209068,-8.61960347647",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1483209068,-8.61960347647


In [58]:
from sklearn.model_selection import train_test_split

# We could totally change this. Utilization of these just probably requires further preprocessing.
ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION = ['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'POLYLINE', 'START_LOCATION']

train_data_sample = train_data.sample(frac=0.2, random_state=420) # frac is used to control percentage of train data used
X = train_data_sample.drop("TRAVEL_TIME", axis=1)
X = X.loc[:, ~X.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
y = train_data_sample["TRAVEL_TIME"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

test_features = test_data.loc[:, ~test_data.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]

In [59]:
from torch.utils.data import DataLoader, Dataset
from mlp_utils import TaxiDataset
    
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_test_public = test_features.astype(float) # This is what we can predict on

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.float32)
X_test_public_tensor = torch.tensor(X_test_public.values, dtype=torch.float32)

train_dataset = TaxiDataset(X_train_tensor, y_train_tensor)
test_dataset = TaxiDataset(X_test_tensor, y_test_tensor)
test_public_dataset = TaxiDataset(X_test_public_tensor)

batch_size = 32

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
dataloader_pred = DataLoader(test_public_dataset, batch_size=batch_size)

In [60]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(13, 128),
      nn.Sigmoid(),
      nn.Linear(128, 64),
      nn.Sigmoid(),
      nn.Linear(64, 32),
      nn.Sigmoid(),
      nn.Linear(32, 1)
    )

  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [61]:
from mlp_utils import num_parameters

mlp = MLP()
if torch.cuda.is_available():
    mlp.cuda()
 
num_parameters(mlp)

12161

In [62]:
from mlp_utils import pipeline
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.001, weight_decay=1e-2)

train_losses, test_loss, predictions = pipeline(mlp, optimizer, dataloader_train=dataloader_train, 
                                                dataloader_test=dataloader_test, dataloader_pred=dataloader_pred,
                        			device=device, criterion=criterion, epochs=10)

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

Training process has finished.


In [63]:
print(f"Final Train Loss: {train_losses[-1]}")
print(f"Test Loss: {test_loss}")

Final Train Loss: 357.2890697286607
Test Loss: 354.1567418286783


In [64]:
from mlp_utils import test_prediction_to_csv

test_prediction_to_csv(predictions, "predicting_five_layer_mlp_call_type_b.csv", test_data)